In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import shap
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score
from datetime import datetime, timedelta

In [2]:
data_dir = "data/PdM/"

df_mst = pd.read_csv(data_dir + "ics_asset_mst.csv", na_values='\\N')
df_sigdata = pd.read_csv(data_dir + "ics_asset_sigdata.csv", na_values='\\N')
df_status_hist = pd.read_csv(data_dir + "ics_asset_status_hist.csv", na_values='\\N')

df_mst.columns = ['ASSET_ID', 'ASSET_NAME', 'SENSOR_NUMBER']
df_sigdata.columns = ['asset_id', 'created_at',	'temperature', 'voltage',
                      'rms_x', 'rms_y', 'rms_z', 'rms_xyz', 'vel_rms_x', 'vel_rms_y', 'vel_rms_z', 'vel_rms_xyz',
                      'skewness_x', 'skewness_y', 'skewness_z', 'vel_skewness_x', 'vel_skewness_y', 'vel_skewness_z',
                      'kurtosis_x', 'kurtosis_y', 'kurtosis_z', 'vel_kurtosis_x', 'vel_kurtosis_y', 'vel_kurtosis_z',
                      'crest_factor_x', 'crest_factor_y', 'crest_factor_z', 'vel_crest_factor_x', 'vel_crest_factor_y', 'vel_crest_factor_z',
                      'peak_x', 'peak_y', 'peak_z', 'vel_peak_x', 'vel_peak_y', 'vel_peak_z',
                      'peak2peak_x', 'peak2peak_y', 'peak2peak_z', 'vel_peak2peak_x', 'vel_peak2peak_y', 'vel_peak2peak_z']
df_status_hist.columns = ['asset_id', 'time', 'imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health', 'asset_health', 'CRT_DT']

In [3]:
# created_at을 datetime으로 변환
df_sigdata['created_at'] = pd.to_datetime(df_sigdata['created_at'], unit='s') + pd.Timedelta(hours=9)
# status_hist의 time을 datetime으로 변환
df_status_hist['time'] = pd.to_datetime(df_status_hist['time'])

In [4]:
# 1월 10일부터 7월 31일까지의 데이터만 사용
df_sigdata = df_sigdata[(df_sigdata['created_at'] >= '2024-01-10') & (df_sigdata['created_at'] <= '2024-07-31')].reset_index(drop=True)
df_status_hist = df_status_hist[(df_status_hist['time'] >= '2024-01-10') & (df_status_hist['time'] <= '2024-07-31')].reset_index(drop=True).drop('CRT_DT', axis=1)

In [5]:
# y_true 뽑아내기
df_sigdata.loc[:, 'time'] = pd.to_datetime(df_sigdata['created_at']).dt.date
df_sigdata['time'] = df_sigdata['time'].astype('datetime64[ns]')
df_result = pd.merge(
    df_sigdata,
    df_status_hist[['asset_id', 'time', 'imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health', 'asset_health']],
    on=['asset_id', 'time'],
    how='left'
)
# pd.isna(x) or
df_result['is_not_healthy'] = df_result['asset_health'].apply(lambda x: x < 0.9)
df_result = df_result['is_not_healthy']
print(df_result)

0        False
1        False
2        False
3        False
4        False
         ...  
96857    False
96858    False
96859    False
96860    False
96861    False
Name: is_not_healthy, Length: 96862, dtype: bool


In [6]:
# 파라미터 분포 설정
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_samples': ['auto', 0.5, 0.75, 1.0],
    'contamination': [0.03, 0.035, 0.04],
    'max_features': [1.0, 0.5, 0.75]
}

# 특성 데이터 선택
X = df_sigdata.drop(columns=['created_at', 'asset_id', 'voltage', 'time'])
repeat = 0

for i in range(4):
    for j in range(4):
        for k in range(3):
            for l in range(3):

                repeat += 1
                # Isolation Forest 모델 적용
                model = IsolationForest(n_estimators=param_dist['n_estimators'][i],
                                        max_samples=param_dist['max_samples'][j],
                                        contamination=param_dist['contamination'][k],
                                        max_features=param_dist['max_features'][l],
                                        random_state=42)
                model.fit(X)

                y_pred = pd.Series(model.predict(X)) != 1
                y_true = df_result
                
                # 정확도, 정밀도, 재현율, F1 스코어 계산
                accuracy = accuracy_score(y_true, y_pred)
                precision = precision_score(y_true, y_pred)
                recall = recall_score(y_true, y_pred)
                f1 = f1_score(y_true, y_pred)

                print(repeat)
                print(param_dist['n_estimators'][i], param_dist['max_samples'][j], param_dist['contamination'][k], param_dist['max_features'][l])
                print(f"Accuracy: {accuracy:.2f}")
                print(f"Precision: {precision:.2f}")
                print(f"Recall: {recall:.2f}")
                print(f"F1 Score: {f1:.2f}")
                print()

1
50 auto 0.03 1.0
Accuracy: 0.91
Precision: 0.54
Recall: 0.17
F1 Score: 0.26

2
50 auto 0.03 0.5
Accuracy: 0.90
Precision: 0.46
Recall: 0.14
F1 Score: 0.22

3
50 auto 0.03 0.75
Accuracy: 0.91
Precision: 0.56
Recall: 0.17
F1 Score: 0.27

4
50 auto 0.035 1.0
Accuracy: 0.90
Precision: 0.50
Recall: 0.18
F1 Score: 0.27

5
50 auto 0.035 0.5
Accuracy: 0.90
Precision: 0.45
Recall: 0.16
F1 Score: 0.24

6
50 auto 0.035 0.75
Accuracy: 0.91
Precision: 0.52
Recall: 0.19
F1 Score: 0.28

7
50 auto 0.04 1.0
Accuracy: 0.90
Precision: 0.47
Recall: 0.20
F1 Score: 0.28

8
50 auto 0.04 0.5
Accuracy: 0.90
Precision: 0.44
Recall: 0.18
F1 Score: 0.26

9
50 auto 0.04 0.75
Accuracy: 0.90
Precision: 0.48
Recall: 0.20
F1 Score: 0.28

10
50 0.5 0.03 1.0
Accuracy: 0.91
Precision: 0.54
Recall: 0.17
F1 Score: 0.26

11
50 0.5 0.03 0.5
Accuracy: 0.91
Precision: 0.53
Recall: 0.17
F1 Score: 0.25

12
50 0.5 0.03 0.75
Accuracy: 0.90
Precision: 0.50
Recall: 0.16
F1 Score: 0.24

13
50 0.5 0.035 1.0
Accuracy: 0.91
Precision: